In [1]:
import json
import pandas as pd
import os

#### Set directories

In [2]:
LABELS_DIR = '/kaggle/input/dentex-challenge-2023/training_data/training_data/quadrant-enumeration-disease/train_quadrant_enumeration_disease.json'
IMG_DIR = '/kaggle/input/dentex-challenge-2023/training_data/training_data/quadrant-enumeration-disease/xrays/'
LABELS_SAVE_PATH = '/kaggle/working/train/labels/'

#### Read json and convert to YOLO format

In [3]:
with open(LABELS_DIR, "r") as json_file:
    config = json.load(json_file)
 
 # Create datafrmae with images
images_df = pd.DataFrame(config['images'])

# Create datafrmae with images
annotations = pd.DataFrame(config['annotations'])

In [4]:
def change_to_yolo(image_df, labels_df, path_to_save):
    
    for image_index, image_row in image_df.iterrows():
        img_id = image_row['id']
        img_width = image_row['width']
        img_height = image_row['height']
        label_filename = path_to_save + image_row['file_name'][:-4] + '.txt'
        
        # Get all labels for img_id
        labels_subset = labels_df[labels_df['image_id'] == img_id]
        
        for label_index, label_row in labels_subset.iterrows():
            bbox = label_row['bbox']
            
            quadrant_id = label_row['category_id_1']
            tooth_id = label_row['category_id_2']
            disease_id = label_row['category_id_3']
            
            # Get x_center, y_center, width, height
            x_center = ((bbox[0] + bbox[2]/2) / img_width) 
            y_center = ((bbox[1] + bbox[3] / 2) / img_height)
            width = bbox[2] / img_width
            height = bbox[3] / img_height
            
            yolo_annotation = f"{quadrant_id} {tooth_id} {disease_id} {x_center} {y_center} {width} {height}"
            
            # quadrant = f"{quadrant_id} {x_center} {y_center} {width} {height}"
            # tooth = f"{tooth_id} {x_center} {y_center} {width} {height}"
            disease = f"{disease_id} {x_center} {y_center} {width} {height}"
            
            with open(label_filename, "a") as w:
                w.write(disease + '\n', )

In [5]:
os.makedirs(LABELS_SAVE_PATH) 
change_to_yolo(images_df, annotations, LABELS_SAVE_PATH)

#### Copy images

In [6]:
import shutil

source_images= '/kaggle/input/dentex-challenge-2023/training_data/training_data/quadrant-enumeration-disease/xrays/'
target_images = '/kaggle/working/train/images/'

shutil.copytree(source_images, target_images)

'/kaggle/working/train/images/'

#### Create validation folder

In [10]:
import os
import shutil
import random

def create_validation_folder(train_images_dir, train_labels_dir, validation_dir, validation_percent):
    # Ensure the validation directory exists, if not, create it
    if not os.path.exists(validation_dir):
        os.makedirs(validation_dir)

    # Create subdirectories for images and labels in the validation directory
    validation_images_dir = os.path.join(validation_dir, 'images')
    validation_labels_dir = os.path.join(validation_dir, 'labels')
    os.makedirs(validation_images_dir, exist_ok=True)
    os.makedirs(validation_labels_dir, exist_ok=True)

    # Get the list of image files in the train images directory
    image_files = os.listdir(train_images_dir)

    # Calculate the number of images to move for validation
    num_images_validation = int(len(image_files) * validation_percent)

    # Select a random subset of image files for validation
    validation_image_files = random.sample(image_files, num_images_validation)

    # Move selected images from train images directory to validation images directory
    for image_file in validation_image_files:
        image_path = os.path.join(train_images_dir, image_file)
        shutil.move(image_path, validation_images_dir)

        # Also move corresponding label file if it exists
        label_file = image_file.split('.')[0] + '.txt'
        if os.path.exists(os.path.join(train_labels_dir, label_file)):
            shutil.move(os.path.join(train_labels_dir, label_file), validation_labels_dir)

# Example usage:
train_images_dir = '/kaggle/working/train/images/'
train_labels_dir = '/kaggle/working/train/labels/'
validation_dir = '/kaggle/working/validation/'
validation_percent = 0.2  # 20 percent validation set

create_validation_folder(train_images_dir, train_labels_dir, validation_dir, validation_percent)


#### Create YAML config file

In [11]:
import yaml

dict_file = {
    'names':{0:'Impacted',
            1:'Caries',
            2:'Periapical Lesion',
            3:'Deep Caries'},
    'path':'/kaggle/working',
    'train': 'train/',
    'val':'validation/'
}


with open(r'config.yaml', 'w') as file:
    documents = yaml.dump(dict_file, file)

#### Enable GPU

In [12]:
import torch

torch.cuda.set_device(0) 

In [13]:
!pip install ultralytics 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.3/721.3 kB 25.0 MB/s eta 0:00:00


In [14]:
#!pip install ultralytics
from ultralytics import YOLO
# Load a model
model = YOLO('yolov8m.pt')  

model.train(data='config.yaml', epochs=15, imgsz=640)


100%|██████████| 49.7M/49.7M [00:00<00:00, 288MB/s]


Ultralytics YOLOv8.1.29 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=config.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_widt

100%|██████████| 755k/755k [00:00<00:00, 41.0MB/s]
2024-03-17 09:22:04,687	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-03-17 09:22:05,552	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-03-17 09:22:07.926557: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 09:22:07.926673: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 09:22:08.058929: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS facto

Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

...


15 epochs completed in 0.149 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train/weights/best.pt, 52.0MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.29 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25842076 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:08<00:00,  1.11s/it]


                   all        253       1280      0.471      0.537      0.472      0.307
              Impacted        253        237        0.8      0.945      0.927      0.577
                Caries        253        774      0.327      0.633       0.38      0.284
     Periapical Lesion        253         55      0.263     0.0364     0.0622     0.0428
           Deep Caries        253        214      0.494      0.533      0.518      0.326
Speed: 0.1ms preprocess, 5.5ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/train


lr/pg0,▂▅▇██▇▇▆▅▅▄▃▂▂▁
lr/pg1,▂▅▇██▇▇▆▅▅▄▃▂▂▁
lr/pg2,▂▅▇██▇▇▆▅▅▄▃▂▂▁
metrics/mAP50(B),▁▃▃▄▄▄▄▆▆▇▇▇███
metrics/mAP50-95(B),▁▃▃▄▃▄▄▅▆▇▇▇███
metrics/precision(B),▅▂█▃▂▅▂▁▃▅▅▄▄▅▅
metrics/recall(B),▁▅▄▃▄▃▄▆▆▇▇▆▇█▆
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▄▄▃▃▃▃▃▂▂▂▂▁▁▁


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x785dc4504460>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0